# Import Libraries

In [2]:
import spacy
from spacy.lang.ur import Urdu
import os
from spacy.tokenizer import Tokenizer
import re 
import string
import pandas as pd
import numpy as np
import math
from scipy import stats


# Load Corpus

In [3]:
#LOAD CORPUS

stopwords = ["‘‘","‘","؟","’’","!", ' ٔ','،','،',',ُِ','%','٪','_','(',')','"','.',':','’','*',',',', ', '', '۔۔۔۔۔۔۔۔'  ]
nlp = Urdu()

f = open("Data\\faiz.txt", "r", encoding="utf-8").read()
g = open("Data\\ghalib.txt", "r", encoding="utf-8").read()
iq = open("Data\\iqbal.txt", "r", encoding="utf-8").read()

# Clean String

In [4]:
#CLEAN CORPUS

sos = 'شش'
eos = 'خخ'
linetokens = [sos,eos]
corpus = f + g + iq

corpus = ''.join(ch for ch in corpus if ch not in stopwords)
corpus = corpus.replace("\n\n\n", '\n')
corpus = corpus.replace("\n\n", '\n')
corpus = corpus.replace("\n ", '\n')
corpus = corpus.replace("\n\n", '\n')
corpus = corpus.replace("\n", ' \n ')
corpus = corpus.replace("\n", 'خخ \n شش')
corpus = corpus.replace(r"''", '')

corpus =''.join(word for word in corpus.split('\n'))

# Tokenize and create Grams

In [5]:
#CREATE TOKENS
def sen_tokenize(string):
    corpus = string.split('  ')  
   
    ll = []
    for num, word in enumerate(corpus): 
        ll.append(word)

    return ll

def word_tokenize(string):
    corpus = string.split(' ')
    
    ll = []
    for num, word in enumerate(corpus): 
        ll.append(word)

    return ll

# Create N-Grams

In [6]:
#CREATE N-Grams

def create_unigram(doc):
    
    result = []
    for word in range(len(doc)-2): #last tokens are garbage
        word = doc[word]
        element = [word]
        result.append(element)
    
    return result


def create_bigram(doc):
    
    result = []
    for word in range(len(doc)-2):
        word1 = doc[word]
        word2 = doc[word+1]
        element = [word1,word2]
        result.append(element)
    
    return result

def create_trigram(doc):
    
    result = []
    for word in range(len(doc)-3):
        word1 = doc[word]
        word2 = doc[word+1]
        word3 = doc[word+2]
        element = [word1,word2,word3]
        result.append(element)
    return result

def backwardgram(doc):
    result = []
    for word in range(len(doc)):
        pass



In [7]:
# uu = [[tokens[i]] for i in range(len(tokens))]
# bb = [[tokens[i], tokens[i+1]] for i in range(len(tokens)-1)]
# tt = [[tokens[i], tokens[i+1],tokens[i+2]] for i in range(len(tokens)-2)]



# Functions for calculating Frequencies and Generation Words

In [8]:
#printing sorted frequencies
def sortdict(dd):
    d = dict()
    for w in sorted(dd, key=dd.get, reverse=True):
        word = w
        d[w] = dd[w]    
    return d


# Calculate frequency for unigrams
def freq_unigram(unigram):
    unifreq = dict()
    for gram in unigram: 
        ss = gram[0]
        if ss in unifreq:
            unifreq[ss] = unifreq[ss] + 1
        else:
            unifreq[ss] = 1
    return unifreq


#Calculate frequency for bigrams
def freq_bigram(bigram):
    bifreq = dict()
    for gram in bigram:
        ss = gram[0] +' '+ gram[1]

        if ss in bifreq:
            bifreq[ss] = bifreq[ss] + 1
        else:
            bifreq[ss] = 1
    return bifreq


def freq_trigram(trigram):
    trifreq = dict()
    for gram in trigram:
        ss = gram[0] +' '+ gram[1] + ' ' + gram[2]

        if ss in trifreq:
            trifreq[ss] = trifreq[ss] + 1
        else:
            trifreq[ss] = 1
    return trifreq


    
def calculate_probability_next_word(freqtrigram, freqbigram, frequnigram, prev_word, word, bigram = True):
    if bigram == True:
        dd = dict()
        for key in freqbigram:
            if key.split(' ')[0] == word:
                freq_bigram = freqbigram[key]
                freq_given = frequnigram[word]
                prob_next_word = freq_bigram / freq_given
                dd[key.split(' ')[1]] = prob_next_word

        return dd
    else:
        dd = dict()
        for key in freqtrigram:
            if key[0] == prev_word and key[1] == word:
                freq_trigram = freqtrigram[key]
                print(freq_trigram)
                x = key[0] + ' ' + key[1]
                print(x)
                freq_given = freqbigram[x]
                print(freq_given)
                prob_next_word = freq_bigram / freq_given
                print(prob_next_word)
                dd[key.split(' ')[2]] = prob_next_word

        return dd

def get_next_word(dd, eos, sos,sentence):
    dd = sortdict(dd)
    for key in dd:
        if key == sos or key == eos or key =='' or key in sentence:
            pass
        else:
            return key

In [9]:
def get_start_words(bigrams, sos):
    start_words = []
    for gram in bigrams:
        
        if gram[0] == sos:
            if gram[1] not in start_words:
                start_words.append(gram[1])
    return start_words

def get_end_words(bigrams,eos):
    end_words = []
    for gram in bigrams:
        if gram[1] == eos:
            if gram[0] not in end_words:
                end_words.append(gram[0])
    return end_words
    
#generate first word:
def get_first_word(start_words,sos,eos):
    x = np.random.randint(1,len(start_words)-2)
    while start_words[x] == sos or start_words[x] == eos or start_words[x] == None:
        
        x = np.random.randint(1,len(start_words)-2)
    else:
        
        return start_words[x]


In [10]:
#generate next word:
def gennextword(word, vocab, freqbigram, frequnigram):
    dd = dict()
    for i in vocab:
        bb = word + ' ' + i
        dd[bb]  = freqbigram[bb]
    dd = sortdict(dd)
    for key in dd:
        next_word = key
        break
    frequnigram = frequnigram
    return next_word



def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

# Call Functions and get Tokens, Grams, Frequencies, list of vocabs

In [11]:
sentences = sen_tokenize(corpus)
tokens = word_tokenize(corpus)



unigram = np.array(create_unigram(tokens))
bigram = np.array(create_bigram(tokens))
trigram = np.array(create_trigram(tokens))
reversegram = [[tokens[len(tokens)-i -1], tokens[len(tokens)-i]] for i in range(1,len(tokens)-2)]
print("done")

frequnigram = freq_unigram(unigram)
freqbigram  = freq_bigram(bigram)
freqtrigram = freq_trigram(trigram)
freqreversegram = freq_bigram(reversegram)
print("done")

start_words = get_start_words(bigram,sos)
end_words = get_end_words(bigram,eos)
first_word = get_first_word(start_words,eos,sos)

vocab = [key for key in frequnigram]
print("done")
print(first_word)

done
done
done
ذلیل


In [12]:
print("number of sentences", len(sentences))
print("number of words", len(tokens))
print("number of unique", len(vocab))
print("number of start words", len(start_words))
print("number of end words", len(end_words))

number of sentences 2958
number of words 32598
number of unique 4474
number of start words 1144
number of end words 795


# BIGRAM GENERATION

In [699]:

for i in range(4):
    for i in range(4):
        stanza = []
        for i in range(4):
            first_word = sos
            word = get_first_word(start_words,sos,eos)
            sentence = []
            sentence.append(first_word)
            sentence.append(word)
            n = np.random.randint(4,8)
            for i in range(n):
                ddd = calculate_probability_next_word(freqtrigram, freqreversegram, frequnigram, first_word, word)
                word = get_next_word(ddd,eos,sos,sentence)
                if not word:
                    word = get_first_word(start_words,sos,eos)
                sentence.append(word)
                if i > 4 and next_word in end_words:
                    break
                    
            sentence.append(eos)
            stanza.append(' '.join(sentence))

        file1 = open("Bigram Generation.txt","a",encoding='utf-8')#write mode 
        for i in stanza:
    #         print(i, "\n")
            file1.write(i) 
            file1.write("\n")
        file1.write("\n")
        file1.close() 

# REVERSE BIGRAM GENERATION

In [700]:
for i in range(4):
    for i in range(4):
        stanza = []
        for i in range(4):
            first_word = sos
            word = get_first_word(start_words,sos,eos)
            sentence = []
            sentence.append(first_word)
            sentence.append(word)
            n = np.random.randint(4,8)
            for i in range(n):
                ddd = calculate_probability_next_word(freqtrigram, freqreversegram, frequnigram, first_word, word)
                word = get_next_word(ddd,eos,sos,sentence)
                if not word:
                    word = get_first_word(start_words,sos,eos)
                sentence.append(word)
                if i > 4 and next_word in end_words:
                    break
                
            sentence.append(eos)
            stanza.append(' '.join(sentence))


        file1 = open("ReverseBigram Generation.txt","a",encoding='utf-8')#write mode 
        for i in stanza:
    #         print(i, "\n")
            file1.write(i) 
            file1.write("\n")
        file1.write("\n")
        file1.close() 

# Trigram generation 

In [701]:
for i in range(4):
    for i in range(4):
        stanza = []
        for i in range(4):
            first_word = sos
            word = get_first_word(start_words,sos,eos)
            sentence = []
            sentence.append(first_word)
            sentence.append(word)
            n = np.random.randint(4,8)
            for i in range(n):
                ddd = calculate_probability_next_word(freqtrigram, freqreversegram, frequnigram, first_word, word, bigram = False)
                next_word = get_next_word(ddd,eos,sos,sentence)
                if not next_word:
                    next_word = get_first_word(start_words,sos,eos)
                sentence.append(next_word)
                first_word = word
                word = next_word
                if i > 4 and next_word in end_words:
                    break
                    
            sentence.append(eos)
            stanza.append(' '.join(sentence))

        file1 = open("Trigram Generation.txt","a",encoding='utf-8')#write mode 
        for i in stanza:
    #         print(i, "\n")
            file1.write(i) 
            file1.write("\n")
        file1.write("\n")
        file1.close() 

# Generation using Laplace Smoothing

In [702]:
def calculate_probability_laplace_smoothing(freqtrigram, freqbigram, frequnigram, prev_word, word, vocab, bigram = True):
    if bigram == True:
        dd = dict()
        for key in freqbigram:
            if key.split(' ')[0] == word:
                freq_bigram = freqbigram[key]
                freq_given = frequnigram[word]
                prob_next_word = (freq_bigram + 1) / (freq_given + len(vocab))
                dd[key.split(' ')[1]] = prob_next_word

        return dd
    else:
        dd = dict()
        for key in freqtrigram:
            if key[0] == prev_word and key[1] == word:
                freq_trigram = freqtrigram[key]
                print(freq_trigram)
                x = key[0] + ' ' + key[1]
                print(x)
                freq_given = freqbigram[x]
                print(freq_given)
                prob_next_word = (freq_bigram + 1) / (freq_given + len(vocab))
                print(prob_next_word)
                dd[key.split(' ')[2]] = prob_next_word

        return dd

# Bigram Generation using Laplace Smoothing

In [703]:
for i in range(4):
    for i in range(4):
        stanza = []
        for i in range(4):
            first_word = sos
            word = get_first_word(start_words,sos,eos)
            sentence = []
            sentence.append(first_word)
            sentence.append(word)
            n = np.random.randint(4,8)
            for i in range(n):
                ddd = calculate_probability_laplace_smoothing(freqtrigram, freqbigram, frequnigram, first_word, word, vocab)
                word = get_next_word(ddd,eos,sos,sentence)
                if not word:
                    word = get_first_word(start_words,sos,eos)
                sentence.append(word)
                if i > 4 and next_word in end_words:
                    break
            sentence.append(eos)
            stanza.append(' '.join(sentence))

        file1 = open("Bigram Generation using Laplace Smoothing.txt","a",encoding='utf-8')#write mode 
        for i in stanza:
    #         print(i, "\n")
            file1.write(i) 
            file1.write("\n")
        file1.write("\n")
        file1.close() 

# Reverse bigram Generation using Laplace Smoothing

In [704]:
for i in range(4):
    for i in range(4):
        stanza = []
        for i in range(4):
            first_word = sos
            word = get_first_word(start_words,sos,eos)
            sentence = []
            sentence.append(first_word)
            sentence.append(word)
            n = np.random.randint(4,8)
            for i in range(n):
                ddd = calculate_probability_laplace_smoothing(freqtrigram, freqreversegram, frequnigram, first_word, word, vocab)
                word = get_next_word(ddd,eos,sos,sentence)
                if not word:
                    word = get_first_word(start_words,sos,eos)
                sentence.append(word)
                if i > 4 and next_word in end_words:
                    break
            sentence.append(eos)
            stanza.append(' '.join(sentence))


        file1 = open("ReverseBigram Generation using Laplace Smoothing.txt","a",encoding='utf-8')#write mode 
        for i in stanza:
    #         print(i, "\n")
            file1.write(i) 
            file1.write("\n")
        file1.write("\n")
        file1.close() 

# Trigram Generation using Laplace Smoothing

In [705]:
for i in range(4):
    for i in range(4):
        stanza = []
        for i in range(4):
            first_word = sos
            word = get_first_word(start_words,sos,eos)
            sentence = []
            sentence.append(first_word)
            sentence.append(word)
            n = np.random.randint(4,8)
            for i in range(n):
                ddd = calculate_probability_laplace_smoothing(freqtrigram, freqbigram, frequnigram, first_word, word, vocab, bigram = False)
                next_word = get_next_word(ddd,eos,sos,sentence)
                if not next_word:
                    next_word = get_first_word(start_words,sos,eos)
                sentence.append(next_word)
                first_word = word
                word = next_word
                if i > 4 and next_word in end_words:
                    break
                    
            sentence.append(eos)
            stanza.append(' '.join(sentence))

    
        file1 = open("Trigram Generation using Laplace Smoothing.txt","a",encoding='utf-8')#write mode 

        for i in stanza:
        #         print(i, "\n")
            file1.write(i) 
            file1.write("\n")
        file1.write("\n")
        file1.close() 

# Bidirectional Gram

In [706]:
def calculate_probability_laplace_smoothing(freqbigram,freqreversegram, frequnigram, prev_word, word, vocab):
    dd = dict()
    for key in freqtrigram:
        if key.split(' ')[1] == word:
            bigram = key.split(' ')[0] + ' ' + key.split(' ')[1]
            reverse = key.split(' ')[2] + ' ' + key.split(' ')[1] 
            freq_bigram = freqbigram[bigram]
            freq_given = frequnigram[word]
            prob_bigram = (freq_bigram + 1) / (freq_given + len(vocab))
            if reverse in freqreversegram.keys():
                freq_reverse = freqreversegram[reverse]
                freq_given = frequnigram[word]
                prob_reverse = (freq_reverse + 1) / (freq_given + len(vocab))
                prob_next_word = prob_bigram + prob_reverse
            
                dd[key.split(' ')[1]] = prob_next_word
            else: 
                dd[key.split(' ')[1]] = prob_bigram

    return dd
   

In [707]:
for i in range(4):
    for i in range(4):
        stanza = []
        for i in range(4):
            first_word = sos
            word = get_first_word(start_words,sos,eos)
            sentence = []
            sentence.append(first_word)
            sentence.append(word)
            n = np.random.randint(4,8)
            for i in range(n):
                ddd = calculate_probability_laplace_smoothing(freqbigram, freqreversegram, frequnigram, first_word, word, vocab)
                next_word = get_next_word(ddd,eos,sos,sentence)
                if not next_word:
                    next_word = get_first_word(start_words,sos,eos)
                
                sentence.append(next_word)
                first_word = word
                word = next_word
                if i > 4 and next_word in end_words:
                    break
                
                    
            sentence.append(eos)
            stanza.append(' '.join(sentence))
        file1 = open("Bidirectional Generation using Laplace Smoothing.txt","a",encoding='utf-8')#write mode 
        for i in stanza:
    #         print(i, "\n")
            file1.write(i) 
            file1.write("\n")
        file1.write("\n")
        file1.close() 

ں
گ
ل
و
ں


In [ ]:
# #Create numpy arrays with indexes = [[words, prob, freq]]

# df1 = pd.DataFrame(sortedunifreq, columns = (['words'],['frequencies']))
# df2 = pd.DataFrame(sorteduniprob, columns = (['words'],['prob']))
# inner_merged = pd.merge(df1, df2)
# uni = np.array(inner_merged)
# # print(len(df1),len(df2))
# # print(len(inner_merged))

# df1 = pd.DataFrame(sortedbifreq, columns = (['words'],['frequencies']))
# df2 = pd.DataFrame(sortedbiprob, columns = (['words'],['prob']))
# inner_merged = pd.merge(df1, df2)
# bi = np.array(inner_merged)
# # print(len(df1),len(df2))
# # print(len(inner_merged))


# df1 = pd.DataFrame(sortedtrifreq, columns = (['words'],['frequencies']))
# df2 = pd.DataFrame(sortedtriprob, columns = (['words'],['prob']))
# inner_merged = pd.merge(df1, df2)
# tri = np.array(inner_merged)
# # print(len(df1),len(df2))
# # print(len(inner_merged))
# print(len(tri))
# print(tri[0][1])
# print(tri[0][2])
    
